### Inverse Design

Inverse design and negative selection are demonstrated here for the $O^{70}$ diblock network phase. The user only needs to do the following:

1) Load in the space group structural files associated with the space groups that form the target and alternate candidate structures.
2) Load a target file. One for the $O^{70}$ structure in the paper is provided. All target structures should be contained in the 'converged_structures' directory. This target must be an integrated $\phi$ field
3) Run inverse design and negative selection using the hyperparameters in the paper. Note: For the $O^{70}$ structure in the paper, the tolerance in free energy difference before stopping is hit is set at 1.0001 (0.01%), here in the default 'run_all' method, 'ns_offset' is set at 1.0005 (0.05%). This may result in convergence to a different sequence than reported in the main text, or even no convergence at all in negative selection, since the relative difference in free energies for the diblock $O^{70}$ phase is very small. Change 'ns_offset' to 1.0001 to use the parameters reported in the paper

The class InverseDesign is contained in src/inverse_design.py and contains the necessary methods. An easy-to-use method called 'run_all' is provided replicating the hyperparameters used in the paper. Additional methods and options can be found in the .py file.


In [ ]:
import sys
import jax
import os

sys.path.append(os.path.abspath(".."))

from src.space_group import SpaceGroupCollection
from src.sequence import Sequence
from src.inverse_design import InverseDesign
from src.utils import STRUCTURES_DIR, CONVERGED_STRUCTURES_DIR, PROJECT_ROOT

jax.config.update("jax_enable_x64", True)  # Enable 64-bit precision

**Load in the space group files**

In [ ]:
### All this can be stored in a csv file and read line by line
# Define the files we want to load
space_groups_to_load = [
    "P_-1_48_1_1",
    "p_6_m_m_48_48_1",
    "I_a_-3_d_32_32_32",
    "I_m_-3_m_32_32_32",
    "F_d_d_d_1_32_32_32",
]  # Replace with actual list of space group names, e.g., ['P_-1_48_1_1']
sg_info_file = PROJECT_ROOT / "space_group_info.csv"  # Load the space group info file

# Set up the space group geometries
# sg_collection contains associated info for each space group. For example, we can access sg_collection.space_groups['P_-1_48_1_1'].cell for the cell parameters
# sg_collection['P_-1_48_1_1'].u0['u_fwd'] access the forward u0 initialization for the space group P_-1_48_1_1
# sg_collection['P_-1_48_1_1'].data is a dictionary that allows access to loaded information files

sg_collection = SpaceGroupCollection.from_files(
    desired_sg=space_groups_to_load,
    sg_info_file=sg_info_file,
    structure_folder=STRUCTURES_DIR,
)

# Set up the sequence - here a multiblock sequence is generated using sigmoid tophat functions.
# Np is number of blocks, Ndim is number of dimensions. Nbasis = Ns for the multiblock and multiblock_taper representations
# No f_basis provided
sequence = Sequence.generate(
    name="inverse_design",
    basis="linear_basis",
    transform="multiblock",
    Nbasis=125,
    Ns=125,
    Np=2,
    Ndim=3,
)

**Automatically run inverse design and negative selection**

In [ ]:
# Initialize an Inverse Design SCFT object
inverse_design = InverseDesign(sg_collection, sequence)

# Load a structure
sg = "F_d_d_d_1_32_32_32"  # Space group of the target structure
file = "id_target_o70.npy"  # File containing the target structure phi field data
target = inverse_design.load_target(
    sg=sg, target_file=file, target_folder=CONVERGED_STRUCTURES_DIR
)

# Alternate space groups to consider for the negative selection
alt_sg = [
    "P_-1_48_1_1",
    "p_6_m_m_48_48_1",
    "I_a_-3_d_32_32_32",
    "I_m_-3_m_32_32_32",
]  # Alternate space groups to consider for the negative selection

# Details for the negative selection
save_name = "id_o70"  # Name to save the result of inverse design
ns_iter = 100  # Number of iterations for the negative selection
ns_offset = 1.0005  # This is the relative energy offset we want to be below the compared energy. 1 means stopping will occur the moment the target structure energy is below that of alternate candidates, while a value greater than 1 means the target free energy must be (ns_offset - 1)% lower than alternates. The default setting is 1.0005. For O70 runs, set as 1.0001 to reproduce the paper results
alt_samples = 6  # Number of alternate structures to sample in the negative selection. There is a tradeoff in computational time and the accuracy of each negative selection step.

# Inverse design + negative selection
inverse_design.run_all(
    target=target,
    alt_sg=alt_sg,
    num_samples_desired=alt_samples,
    save_name=save_name,
    ns_iter=ns_iter,
    ns_offset=ns_offset,
)